In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
from medvqa.models.huggingface_utils import (
    CachedTextEmbeddingExtractor,
    SupportedHuggingfaceMedicalBERTModels,
)
from medvqa.utils.files import (
    load_jsonl, load_pickle,
)
from medvqa.utils.math import rank_vectors_by_dot_product
import numpy as np

In [3]:
from importlib import reload
import medvqa

In [425]:
reload(medvqa.utils.constants)

<module 'medvqa.utils.constants' from '/home/pamessina/medvqa/medvqa/utils/constants.py'>

In [4]:
integrated_fact_metadata_filepath = "/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl"

In [5]:
from medvqa.utils.constants import MULTIDATASET_UNIFIED_CLASSES
phrases = MULTIDATASET_UNIFIED_CLASSES
print(len(phrases))

297


In [6]:
# Create embedding extractor
embedding_extractor = CachedTextEmbeddingExtractor(
    model_name='microsoft/BiomedVLP-CXR-BERT-specialized',
    model_checkpoint_folder_path='/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)',
    device='cuda',
)

checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']


In [7]:
# Load integrated fact metadata
print('Loading integrated fact metadata from:', integrated_fact_metadata_filepath)
integrated_fact_metadata = load_jsonl(integrated_fact_metadata_filepath)

Loading integrated fact metadata from: /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl


In [8]:
# Extract facts
all_facts = set()
for x in integrated_fact_metadata:
    fact = x['fact']
    if fact: all_facts.add(fact)
all_facts = list(all_facts)
all_facts.sort()
print(f'Loaded {len(all_facts)} facts.')

Loaded 591920 facts.


In [9]:
fact_embeddings = embedding_extractor.compute_text_embeddings(all_facts)

  0%|                                                                                        | 0/591920 [00:00<?, ?it/s]

Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["hashes"]) = 876541
self.cache["embeddings"].shape = (876541, 128)


100%|███████████████████████████████████████████████████████████████████████| 591920/591920 [00:01<00:00, 314964.84it/s]


In [10]:
phrase_embeddings = embedding_extractor.compute_text_embeddings(phrases)

100%|█████████████████████████████████████████████████████████████████████████████| 297/297 [00:00<00:00, 261264.32it/s]


In [11]:
i = phrases.index('bullae')
i

38

In [12]:
idxs = rank_vectors_by_dot_product(fact_embeddings, phrase_embeddings[i])

In [14]:
for k in range(100):
    print(all_facts[idxs[k]], np.dot(fact_embeddings[idxs[k]], phrase_embeddings[i]))

bullae 0.99999994
bulla 0.9935278
bulla formation 0.990708
blebs 0.9893153
bleb 0.988828
bullous changes 0.98770356
bullous change 0.9871189
cysts 0.9870987
cystic changes 0.97311807
cystic abnormalities 0.96693194
paraseptal bullae 0.9661053
blebectomy 0.9629918
cystic lucencies 0.96020746
unusual bulla 0.95713097
pneumatocele 0.9561011
areas of bullous change 0.9550576
sebaceous cyst 0.9536319
underlying bleb 0.95282066
multiple bullae 0.9515826
bullous disease 0.95083356
multiple bulla 0.9505608
pneumatocele formation 0.94924504
PROMINENT bullous change 0.94900554
bipical blebs 0.94535685
bullae in the lungs 0.9449865
prominent bullous formation in the region 0.9439609
pneumatoceles 0.9427241
multiple blebs 0.9425837
seed cysts 0.93939626
known bleb in the region 0.93627673
bullae filled with fluid 0.93498886
bulla within the lungs 0.93294966
bulla formation in the lungs 0.9322265
evidence of bullous disease 0.93140024
bullas in the lungs 0.93029946
several large bullae 0.9298261
bu

In [340]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_facts_similar_to_anchor_facts" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(606074,61621903).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--min_similarity_threshold 0.65

Running _find_facts_similar_to_anchor_facts()...
len(anchor_facts): 297
min_similarity_threshold: 0.65
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_fact_metadata(606074,61621903).jsonl...
Loaded 505377 facts (skipped 100693 facts)
len(facts) before removing invalid facts: 505377
100%|█████████████████████████████████| 505377/505377 [01:05<00:00, 7694.00it/s]
len(facts) after removing invalid facts: 503311
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                | 0/503311 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["hashes"]) = 702198
self.cache["embeddings"].shape = (702198, 128)
100%|█████████████████████████████████████| 297/297 [00:00<00:00, 208521.64it/s]
fact_embedd

In [136]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_facts_similar_to_anchor_facts" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl"\
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--min_similarity_threshold 0.5

Running _find_facts_similar_to_anchor_facts()...
len(anchor_facts): 297
min_similarity_threshold: 0.5
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl...
len(banned_facts): 112282
Loaded 479638 unique facts (skipped 112282 facts)
Loaded 250466 unique general facts
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                | 0/479638 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["hashes"]) = 876541
self.cache["embeddings"].shape = (876541, 128)
100%|█████████████████████████████████████| 297/297 [00:00<00:00, 157604.79it/s]
unique_fact_embeddings.shape: (479638, 128)
unique_general_fact_embeddings.shape: (250466, 128)
anchor_fact_e

In [2]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_facts_similar_to_anchor_facts" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl"\
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--min_similarity_threshold 0.6

Running _find_facts_similar_to_anchor_facts()...
len(anchor_facts): 297
min_similarity_threshold: 0.6
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl...
Loaded 479638 unique facts (skipped 112282 facts)
Loaded 252372 unique general facts
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                | 0/479638 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["hashes"]) = 855787
self.cache["embeddings"].shape = (855787, 128)
100%|█████████████████████████████████████| 297/297 [00:00<00:00, 260881.32it/s]
unique_fact_embeddings.shape: (479638, 128)
unique_general_fact_embeddings.shape: (252372, 128)
anchor_fact_embeddings.shape: (297, 128

In [33]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_facts_similar_to_anchor_facts" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--min_similarity_threshold 0.7

Running _find_facts_similar_to_anchor_facts()...
len(anchor_facts): 297
min_similarity_threshold: 0.7
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl...
Loaded 479638 unique facts (skipped 112282 facts)
Loaded 252372 unique general facts
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                | 0/479638 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=469,680486110513564934).pkl
len(self.cache["hashes"]) = 855787
self.cache["embeddings"].shape = (855787, 128)
100%|█████████████████████████████████████| 297/297 [00:00<00:00, 264089.10it/s]
unique_fact_embeddings.shape: (479638, 128)
unique_general_fact_embeddings.shape: (252372, 128)
anchor_fact_embeddings.shape: (297, 128

In [149]:
!ls -lh "/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=297,sf=416649,sgf=207909,sim_th=0.500)(hash=635,1389408443551332457).pkl"

-rw-rw-r-- 1 pamessina socvis 42M Jan  7 16:41 '/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=297,sf=416649,sgf=207909,sim_th=0.500)(hash=635,1389408443551332457).pkl'


In [137]:
tmp = load_pickle("/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=297,sf=416649,sgf=207909,sim_th=0.500)(hash=635,1389408443551332457).pkl")

In [138]:
tmp.keys()

dict_keys(['anchor_facts', 'preserved_general_facts', 'preserved_facts', 'most_similar_to_fact', 'most_similar_to_general_fact'])

In [142]:
tmp['preserved_facts'].index('pneumothorax')

265832

In [143]:
class_idx = tmp['anchor_facts'].index('reticulation')
class_idx

252

In [144]:
idxs1 = [i for i, x in enumerate(tmp['most_similar_to_general_fact']) if x is not None and x[1] == class_idx]
idxs1.sort(key=lambda idx: tmp['most_similar_to_general_fact'][idx][0], reverse=True)
idxs2 = [i for i, x in enumerate(tmp['most_similar_to_fact']) if x is not None and x[1] == class_idx]
idxs2.sort(key=lambda idx: tmp['most_similar_to_fact'][idx][0], reverse=True)
len(idxs1), len(idxs2)

(42, 33)

In [145]:
for i in range(0, min(200, len(idxs1))):
    i = idxs1[i]
    anc = tmp['anchor_facts'][tmp['most_similar_to_general_fact'][i][1]]
    fac = tmp['preserved_general_facts'][i]
    sim = tmp['most_similar_to_general_fact'][i][0]
    print(f'{sim}\t{fac} ({anc})')

0.9706587791442871	reticular prominence (reticulation)
0.970277726650238	reticulation seen (reticulation)
0.9595723748207092	background reticulation (reticulation)
0.958702564239502	reticulation pattern observed (reticulation)
0.9443132877349854	conspicuity of reticular architecture (reticulation)
0.9388192892074585	coarse reticulation (reticulation)
0.9381308555603027	prominent reticular pattern (reticulation)
0.9291142225265503	coarse reticulations (reticulation)
0.9289706945419312	interstitial abnormality with B-lines (reticulation)
0.9217033386230469	interstitial abnormality including B-lines (reticulation)
0.9206329584121704	cortical reticulation (reticulation)
0.9119927883148193	reticulation most pronounced (reticulation)
0.9075450301170349	generalized reticulation (reticulation)
0.9017454981803894	dense streak (reticulation)
0.8959839344024658	coarse markings (reticulation)
0.8754942417144775	new reticular pattern (reticulation)
0.8647550344467163	coarsened markings (reticulatio

In [146]:
for i in range(0, min(200, len(idxs2))):
    i = idxs2[i]
    anc = tmp['anchor_facts'][tmp['most_similar_to_fact'][i][1]]
    fac = tmp['preserved_facts'][i]
    sim = tmp['most_similar_to_fact'][i][0]
    print(f'{sim}\t{fac} ({anc})')

0.9797871708869934	reticulations (reticulation)
0.9560151100158691	superimposed reticulation (reticulation)
0.9431335926055908	reticular prominence process (reticulation)
0.9428203701972961	conspicuity of the reticular architecture (reticulation)
0.9388192892074585	coarse reticulation (reticulation)
0.9289706945419312	interstitial abnormality with B-lines (reticulation)
0.9217033386230469	interstitial abnormality including B-lines (reticulation)
0.8746028542518616	interstitial abnormality best seen on the lateral view (reticulation)
0.8647550344467163	coarsened markings (reticulation)
0.8438680171966553	prominent tissue markings (reticulation)
0.838477373123169	intraparenchymal abnormalities (reticulation)
0.8379271030426025	calcified parenchymal changes (reticulation)
0.8340967893600464	intraparenchymal abnormality (reticulation)
0.804874062538147	B-lines compared with the prior radiograph (reticulation)
0.7993770241737366	diffuse interstitial abnormality in the lateral view (reticula

In [443]:
for x in integrated_fact_metadata:
    if x['fact'] == "right lower lobe consolidation":
        print(x)
        break

{'fact': 'right lower lobe consolidation', 'metadata': {'anatomical location': 'right lower lobe', 'detailed observation': 'consolidation', 'short observation': 'consolidation', 'category': 'anatomical finding', 'health status': 'abnormal', 'prev_study_comparison?': 'no', 'comparison status': ''}, 'extraction_method': 'gpt-3.5-turbo-0613'}


In [150]:
!python ../../scripts/mimiccxr/find_pos_neg_facts_per_report_around_reference_classes.py \
--task "find_positive_negative_facts_per_report_with_fact_embeddings_and_mlp_nli" \
--integrated_report_facts_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl" \
--integrated_fact_metadata_jsonl_filepath \
"/home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl" \
--facts_similar_to_anchor_facts_pickle_filepath \
"/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=297,sf=416649,sgf=207909,sim_th=0.500)(hash=635,1389408443551332457).pkl" \
--fact_embedding_model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--fact_embedding_model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240629_084405_MIMIC-CXR(triplets+classif+entcont+nli+radgraph+autoencoder)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--fact_embedding_batch_size 200 \
--fact_embedding_num_workers 4 \
--mlp_batch_size 1000 \
--mlp_num_workers 4 \
--mlp_nli_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20240731_002153_NLI(Embedding-based)_EmbeddingBasedNLI(128,512-256-128)" \
--mlp_nli_entailment_threshold 0.1723824292421341 \
--num_clusters 300 \
--max_negative_facts_per_report 500

Running _find_positive_negative_facts_per_report_with_fact_embeddings_and_mlp_nli()...
Reading /home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl...
n_reports: 227835
len(all_facts): 586352
Loading /home/pamessina/medvqa-workspace/cache/mimiccxr/huggingface/integrated_fact_metadata(591920,209004293).jsonl...
len(all_facts): 769532
Loading /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_find_facts_similar_to_anchor_facts(af=297,sf=416649,sgf=207909,sim_th=0.500)(hash=635,1389408443551332457).pkl...
len(preserved_facts): 416649
len(preserved_general_facts): 207909
len(all_facts): 769532
checkpoint_names = ['checkpoint_196_cacc+chf1+chf1+cscc+encc+hscc+nlcc+sass+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9189.pt']
  0%|                                                | 0/769532 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/

In [15]:
!ls -lh "/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_sim_facts=550410,num_clusters=300,max_neg=500,ent_th=0.172)(hash=997,752767386763313321).pkl"

-rw-rw-r-- 1 pamessina socvis 908M Jan  7 17:31 '/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_sim_facts=550410,num_clusters=300,max_neg=500,ent_th=0.172)(hash=997,752767386763313321).pkl'


In [16]:
aux = load_pickle('/mnt/data/pamessina_folder_backup_15_10_24/pamessina/medvqa-workspace/cache/mimiccxr_dicom_id_to_pos_neg_facts(num_sim_facts=550410,num_clusters=300,max_neg=500,ent_th=0.172)(hash=997,752767386763313321).pkl')

In [17]:
aux.keys()

dict_keys(['facts', 'embeddings', 'dicom_id_to_pos_neg_facts'])

In [18]:
len(aux['facts'])

769532

In [19]:
aux['embeddings'].shape

(769532, 128)

In [20]:
dicom_id_to_pos_neg_facts = aux['dicom_id_to_pos_neg_facts']

In [21]:
dicom_ids = list(dicom_id_to_pos_neg_facts.keys())

In [22]:
integrated_report_facts = load_jsonl('/home/pamessina/medvqa-workspace/cache/mimiccxr/integrated_report_facts(79040140).jsonl')

In [23]:
from medvqa.datasets.mimiccxr import get_detailed_metadata_for_dicom_id
from medvqa.utils.files import read_txt

In [62]:
dicom_id = dicom_ids[-3]
metadata = get_detailed_metadata_for_dicom_id(dicom_id)[0]
print(metadata)
print()
print()
print(read_txt(metadata['filepath']))
print()
print()
print('All facts:')
facts = integrated_report_facts[metadata['report_index']]['facts']
print(facts)
print()
print()
print('positive facts:')
positive_facts = [aux['facts'][idx] for idx in dicom_id_to_pos_neg_facts[dicom_id][0]]
print(positive_facts)
print()
print()
print('excluded positive facts:')
print([f for f in facts if f not in positive_facts])
print()
print()
print('negative facts:')
for idx in dicom_id_to_pos_neg_facts[dicom_id][1]:
    print(aux['facts'][idx])

{'report_index': 227833, 'part_id': 19, 'subject_id': 19095472, 'study_id': 55025385, 'dicom_id': 'f31268a9-0df36c66-69874c18-f1929147-ea49bde2', 'view_pos': 'PA', 'split': 'train', 'filepath': '/mnt/data/mimic-cxr/files/p19/p19095472/s55025385.txt', 'dicom_id_view_pos_pairs': [('baca4b7b-efde034e-c5a89f7d-dad69eb1-05062cd9', 'LL'), ('f31268a9-0df36c66-69874c18-f1929147-ea49bde2', 'PA')]}


                                 FINAL REPORT
 EXAMINATION:  CHEST (PA AND LAT)
 
 INDICATION:  ___ year old woman with mitral valve endocarditis with persistent
 fevers  // evidence of infiltrate
 
 TECHNIQUE:  Chest PA and lateral
 
 COMPARISON:  ___
 
 FINDINGS: 
 
 Cardiomediastinal contours are normal.  Opacities in the left base have
 resolved. There is no pneumothorax or pleural effusion.  The osseous
 structures are unremarkable.  Right PICC tip remains in standard position.
 
 IMPRESSION: 
 
 No acute cardiopulmonary abnormalities



All facts:
['normal cardiomediastinal contours', 'resolve

In [118]:
integrated_fact_metadata[0]

{'fact': 'history of sternal debridement in bilateral pectoralis flaps',
 'metadata': {'reason': 'The statement refers to a medical history involving surgical procedures but does not describe a current observation from a radiology report. It indicates past interventions rather than findings from an imaging study.',
  'too_noisy_or_irrelevant': 'yes',
  'visually_observable': 'no',
  'category': 'does not apply',
  'abnormality_status': 'does not apply',
  'anatomical_location': 'does not apply',
  'general_observation': 'does not apply'},
 'extraction_method': 'gpt-4o-mini'}

In [30]:
from medvqa.scripts.mimiccxr.find_pos_neg_facts_per_report_around_reference_classes import _filter_facts

In [31]:
_unique_facts, _unique_general_facts, _f2gf, banned_facts = _filter_facts(integrated_fact_metadata)

len(banned_facts): 112282
Loaded 479638 unique facts (skipped 112282 facts)
Loaded 250466 unique general facts


In [33]:
banned_facts = list(banned_facts)

In [39]:
banned_facts[2]

'no superadded pneumonia'

In [46]:
b_idxs = [i for i,x in enumerate(banned_facts) if 'line' in x.lower()]
len(b_idxs)

1813

In [54]:
query = 'possible artifactual at the right lung base'
# query = banned_facts[b_idxs[10]]
print(query)
print()

for x in integrated_fact_metadata:
    if x['fact'] == query:
        print(x)
        break
print()

q_idx = all_facts.index(query)
sims = np.dot(phrase_embeddings, fact_embeddings[q_idx])
idxs = np.argsort(sims)[::-1]
for idx in idxs:
    print(f'{sims[idx]}  {phrases[idx]}')

possible artifactual at the right lung base

{'fact': 'possible artifactual at the right lung base', 'metadata': {'reason': 'The statement suggests the presence of an artifact in the imaging of the right lung base. Artifacts can affect the interpretation of radiological images and are relevant observations in radiology reports.', 'too_noisy_or_irrelevant': 'no', 'visually_observable': 'yes', 'category': 'technical assessment', 'abnormality_status': 'does not apply', 'anatomical_location': 'right lung base', 'general_observation': 'possible artifactual at the lung base'}, 'extraction_method': 'gpt-4o-mini'}

0.4726397693157196  prominent hila
0.46213287115097046  prominence of hilum
0.4599626064300537  hyperinflation
0.4561922252178192  hilum prominent
0.4299100637435913  hyperaeration
0.42224496603012085  hyperexpanded lungs
0.4140942692756653  lobar/segmental collapse
0.3989369869232178  interstitial opacification
0.3954535722732544  increased interstitial markings
0.3942326605319977 

In [130]:
query_idxs = []
for i, x in enumerate(integrated_fact_metadata):
    if x['metadata']['general_observation'] == query:
        query_idxs.append(i)
        if x['metadata']['category'] not in ('anatomical finding', 'disease'):
            print(x)
            print()
len(query_idxs)

{'fact': 'no pneumothorax after pacemaker placement', 'metadata': {'reason': 'The statement indicates the absence of a pneumothorax, which is a significant observation following a medical procedure. This finding can be visually confirmed on a chest X-ray. It suggests that the procedure was successful without complications.', 'too_noisy_or_irrelevant': 'no', 'visually_observable': 'yes', 'category': 'procedure', 'abnormality_status': 'completely normal or healthy', 'anatomical_location': 'chest', 'general_observation': 'no pneumothorax'}, 'extraction_method': 't5-small-finetuned'}

{'fact': 'no pneumothorax after pigtail placement', 'metadata': {'reason': "The statement indicates the absence of a pneumothorax, which is a significant observation following a medical procedure. It suggests that the pigtail placement of the device has been performed, which is relevant to the assessment of the patient's condition. This finding can be visually confirmed on a chest X-ray.", 'too_noisy_or_irrel

245

In [439]:
from collections import Counter

In [440]:
Counter(x['metadata']['category'] for x in integrated_fact_metadata)

Counter({'anatomical finding': 396065,
         'disease': 37122,
         'technical assessment': 53424,
         'tubes and lines': 83396,
         'device': 35156,
         'procedure': 160,
         'unknown': 83,
         'health status': 14,
         'symptom': 226,
         '': 413,
         'foreign body': 5,
         'surgical procedure': 3,
         'complication': 2,
         'treatment': 1,
         'surgery': 2,
         'normal': 1,
         'tubes and lines unchanged': 1})

In [441]:
Counter(x['metadata']['health status'] for x in integrated_fact_metadata)

Counter({'ambiguous': 41261,
         'abnormal': 315390,
         'unknown': 172249,
         'normal': 76889,
         'resolved': 5,
         'improved': 23,
         '': 257})

In [471]:
# _VALID_JSON_OBJECT_REGEX = re.compile(r"\{\s*\"reason\"\s*:\s*\"[^\"]*\"\s*,\s*\"too\_noisy\_or\_irrelevant\"\s*:\s*\"[^\"]*\"\s*,\s*\"visually\_observable\"\s*:\s*\"[^\"]*\"\s*,\s*\"category\"\s*:\s*\"[^\"]*\"\s*,\s*\"abnormality\_status\"\s*:\s*\"[^\"]*\"\s*,\s*\"anatomical\_location\?\"\s*:\s*\"[^\"]*\"\s*,\s*\"general\_observation\"\s*:\s*\"[^\"]*\"\s*\}")

_VALID_JSON_OBJECT_REGEX = re.compile(
    r'\{\s*"reason"\s*:\s*"[^"]*"\s*,\s*"too_noisy_or_irrelevant"\s*:\s*"[^"]*"\s*,\s*"visually_observable"\s*:\s*"[^"]*"\s*,\s*"category"\s*:\s*"[^"]*"\s*,\s*"abnormality_status"\s*:\s*"[^"]*"\s*,\s*"anatomical_location"\s*:\s*"[^"]*"\s*,\s*"general_observation"\s*:\s*"[^"]*"\s*\}',
    re.DOTALL
)

In [483]:
match = _VALID_JSON_OBJECT_REGEX.search("""{
  "reason": "The statement describes the presence of surgical clips, which are a visual finding on a chest X-ray. These clips indicate a previous surgical procedure, specifically an axillary dissection. This information is relevant for assessing the patient's surgical history and any potential implications for current health.",
  "too_noisy_or_irrelevant": "no",
  "visually_observable": "yes",
  "category": "device",
  "abnormality_status": "does not apply",
  "anatomical_location": "axilla",
  "general_observation": "surgical clips present"
}""")
if match:
    print(match.group(0))

{
  "reason": "The statement describes the presence of surgical clips, which are a visual finding on a chest X-ray. These clips indicate a previous surgical procedure, specifically an axillary dissection. This information is relevant for assessing the patient's surgical history and any potential implications for current health.",
  "too_noisy_or_irrelevant": "no",
  "visually_observable": "yes",
  "category": "device",
  "abnormality_status": "does not apply",
  "anatomical_location": "axilla",
  "general_observation": "surgical clips present"
}
